In [1]:
import re
import psycopg2
import yaml
from contextlib import closing

import numpy as np
import pandas as pd

In [2]:
random_state = np.random.RandomState(42)

In [3]:
with open('config.yaml', 'r') as config_file:
    config = yaml.load(config_file, Loader=yaml.Loader)

In [4]:
conn = psycopg2.connect(**config['db'])

In [5]:
with closing(conn.cursor()) as c:
    c.execute('''
        SELECT at.id, at.name, count(*) photos
        FROM aircraft_photos ap
        LEFT JOIN aircraft_types at on ap.type_id = at.id
        GROUP BY (at.id, at.name)
        ORDER BY photos DESC
    ''')
    
    df_stats = pd.DataFrame(c.fetchall(), columns=['id', 'name', 'photos'])
    
conn.commit()

In [6]:
df_stats

,id,name,photos
0,3056,Boeing 737-800 (BBJ2/C-40/P-8),500
1,966,Airbus A330-300,500
2,6873,McDonnell Douglas MD-11,499
3,6776,McDonnell Douglas MD-80 (DC-9-80),499
4,963,Airbus A321,495
5,962,Airbus A320,494
6,3069,Boeing 757-200 (C-32),493
7,965,Airbus A330-200,491
8,2978,Boeing 777-300,489
9,3064,Boeing 747-400 (AL-1),487


In [7]:
with closing(conn.cursor()) as c:
    c.execute('''
        SELECT
            (SELECT at.name FROM aircraft_types at WHERE at.id = ap.type_id) AS name,
            ap.path
        FROM aircraft_photos ap
    ''')
    
    df_images = pd.DataFrame(c.fetchall(), columns=['name', 'path']).sample(frac=1, random_state=random_state)
    
conn.commit()

In [8]:
df_images

,name,path
2471,Boeing 747-200 (C-25/E-4),full/069c7d9c0a2e8cacdbd3dd6fbe28c9b28d50878d.jpg
7188,Airbus A330-200,full/7bc65788b7b9b0c12908d612a62575fc3582f36c.jpg
6997,Airbus A330-300,full/6bc6f27f6f8c32bb6ffc9211f5a8eac860e2e763.jpg
5391,Boeing 737-400,full/9514715985036c821ca10bfdbd7d1710b60e4237.jpg
1220,Boeing 747-400 (AL-1),full/6e2b66e4289e1de6a888efea37b1c995955ac67d.jpg
...,...,...
5734,Boeing 737-700 (BBJ/C-40 Clipper),full/54647434054d673c44fb32e49adca81a9be49dac.jpg
5191,Boeing 777-300,full/27634f76c5114a55bb4758701e083abf9a247bf8.jpg
5390,Boeing 737-400,full/b769ec6eab7de0146c69017198bceefffe5f6a4e.jpg
860,Airbus A330-300,full/3d2bb51710bda097899393d86a0a05318f9403fa.jpg


In [9]:
pd.DataFrame({'name': np.sort(df_images['name'].unique())})

,name
0,Airbus A319
1,Airbus A320
2,Airbus A321
3,Airbus A330-200
4,Airbus A330-300
5,Airbus A340-300
6,Boeing 737-200 (T-43)
7,Boeing 737-300
8,Boeing 737-400
9,Boeing 737-500


In [10]:
list(np.sort(df_images['name'].unique()))

['Airbus A319',
 'Airbus A320',
 'Airbus A321',
 'Airbus A330-200',
 'Airbus A330-300',
 'Airbus A340-300',
 'Boeing 737-200 (T-43)',
 'Boeing 737-300',
 'Boeing 737-400',
 'Boeing 737-500',
 'Boeing 737-700 (BBJ/C-40 Clipper)',
 'Boeing 737-800 (BBJ2/C-40/P-8)',
 'Boeing 747-200 (C-25/E-4)',
 'Boeing 747-400 (AL-1)',
 'Boeing 757-200 (C-32)',
 'Boeing 767-300',
 'Boeing 777-200',
 'Boeing 777-300',
 'McDonnell Douglas MD-11',
 'McDonnell Douglas MD-80 (DC-9-80)']

In [11]:
airbus_pattern = re.compile(r'Airbus A?(?P<major>\d*)(?P<minor>-\d*)?\s*(\(.*\))?\Z')
boeing_pattern = re.compile(r'Boeing B?(?P<major>\d*)(?P<minor>-\d*)?\s*(\(.*\))?\Z')
md_pattern = re.compile(r'McDonnell Douglas MD-(?P<major>\d*)\s*(\(.*\))?\Z')

def codename(str):
    m = airbus_pattern.match(str)
    if m is not None:
        major = m.group('major')
        minor = '' if m.group('minor') is None else m.group('minor')
        return f'A{major}{minor}'
    
    
    m = boeing_pattern.match(str)
    if m is not None:
        major = m.group('major')
        minor = '' if m.group('minor') is None else m.group('minor')
        return f'B{major}{minor}'
    
    
    m = md_pattern.match(str)
    if m is not None:
        major = m.group('major')
        return f'MD-{major}'
    
    raise Exception(f'Unexpected aircraft type <{str}>')

In [12]:
df_stats['name'] = df_stats['name'].map(codename)
df_stats

,id,name,photos
0,3056,B737-800,500
1,966,A330-300,500
2,6873,MD-11,499
3,6776,MD-80,499
4,963,A321,495
5,962,A320,494
6,3069,B757-200,493
7,965,A330-200,491
8,2978,B777-300,489
9,3064,B747-400,487


In [13]:
df_images['name'] = df_images['name'].map(codename)
df_images

,name,path
2471,B747-200,full/069c7d9c0a2e8cacdbd3dd6fbe28c9b28d50878d.jpg
7188,A330-200,full/7bc65788b7b9b0c12908d612a62575fc3582f36c.jpg
6997,A330-300,full/6bc6f27f6f8c32bb6ffc9211f5a8eac860e2e763.jpg
5391,B737-400,full/9514715985036c821ca10bfdbd7d1710b60e4237.jpg
1220,B747-400,full/6e2b66e4289e1de6a888efea37b1c995955ac67d.jpg
...,...,...
5734,B737-700,full/54647434054d673c44fb32e49adca81a9be49dac.jpg
5191,B777-300,full/27634f76c5114a55bb4758701e083abf9a247bf8.jpg
5390,B737-400,full/b769ec6eab7de0146c69017198bceefffe5f6a4e.jpg
860,A330-300,full/3d2bb51710bda097899393d86a0a05318f9403fa.jpg


In [14]:
list(np.sort(df_images['name'].unique()))

['A319',
 'A320',
 'A321',
 'A330-200',
 'A330-300',
 'A340-300',
 'B737-200',
 'B737-300',
 'B737-400',
 'B737-500',
 'B737-700',
 'B737-800',
 'B747-200',
 'B747-400',
 'B757-200',
 'B767-300',
 'B777-200',
 'B777-300',
 'MD-11',
 'MD-80']